Imports

In [18]:
import os 
import pandas as pd
import numpy as np 
from PIL import Image
import tensorflow as tf

Build our X and y with all the data we need 

In [2]:
directory = '../raw_data/faces/' 
folders = [x[0] for x in os.walk(directory)][1:]  #ignore the first folder which is the root dir

In [75]:
emotions = {'anger': 0, 
            'contempt': 1, 
            'disgust': 2,
            'fear': 3,
            'happiness': 4,
            'neutrality': 5,
            'sadness': 6, 
            'surprise': 7}

NameError: name 'dups' is not defined

In [89]:
from tensorflow.keras.utils import to_categorical

data = []
emotions_enum = []
for folder in folders: 
    for filename in os.listdir(folder): 
        emotion = os.path.basename(folder) #get folder name 
        f = os.path.join(folder, filename)
        data.append([f, filename, emotions[emotion]])
    

In [90]:
df = pd.DataFrame(data, columns=('path','file_name', 'emotion'))
df = df.drop_duplicates(subset = ['file_name'], keep=False)
files = list(df['path'])
labels = list(df['emotion'])

In [91]:
ims = []

for f in files:
    if os.path.isfile(f):
        im = Image.open(f)
        im = np.expand_dims(np.array(im), axis=2)
        ims.append(im)

In [92]:
num_classes = len(set(labels))
y = to_categorical(labels, num_classes)
X = np.array(ims)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

first_split = int(len(X) /6.)
second_split = first_split + int(len(X) * 0.2)
X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]

In [94]:
X_test.shape

(851, 224, 224, 1)

In [95]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_model():

    model = Sequential()

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu', input_shape=(224, 224, 1)))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(8, activation='softmax'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [71]:
model_ = load_model()
model_.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 215, 215, 16)      1616      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 71, 71, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 64, 64, 32)        32800     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 21, 21, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 32)        36896     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 5, 5, 32)       

In [96]:
es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model_.fit(X_train, y_train,
                             validation_data = (X_val, y_val),
                             batch_size = 16, 
                             epochs = 100, 
                             callbacks=[es])

Epoch 1/100
203/203 [==============================] - 33s 160ms/step - loss: 0.9846 - accuracy: 0.7213 - val_loss: 1.0740 - val_accuracy: 0.6742
Epoch 2/100
203/203 [==============================] - 51s 249ms/step - loss: 0.5699 - accuracy: 0.8214 - val_loss: 0.7146 - val_accuracy: 0.7975
Epoch 3/100
203/203 [==============================] - 43s 212ms/step - loss: 0.3767 - accuracy: 0.8854 - val_loss: 0.6544 - val_accuracy: 0.8236
Epoch 4/100
203/203 [==============================] - 48s 236ms/step - loss: 0.2739 - accuracy: 0.9184 - val_loss: 0.6578 - val_accuracy: 0.8290
Epoch 5/100
203/203 [==============================] - 32s 159ms/step - loss: 0.1777 - accuracy: 0.9509 - val_loss: 0.6448 - val_accuracy: 0.8479
Epoch 6/100
203/203 [==============================] - 46s 228ms/step - loss: 0.1328 - accuracy: 0.9710 - val_loss: 0.6572 - val_accuracy: 0.8443
Epoch 7/100
203/203 [==============================] - 34s 166ms/step - loss: 0.1079 - accuracy: 0.9759 - val_loss: 0.5478 -

In [97]:
model_.evaluate(X_test, y_test)

27/27 [==============================] - 2s 57ms/step - loss: 1.0327 - accuracy: 0.7685


[1.032728672027588, 0.7685076594352722]